In [1]:
using LinearAlgebra, CairoMakie, BenchmarkTools

In [40]:
# function householder(v::Vector{T}) where T<:Number
#     N = sizeof(v)[1]
#     vv = normalize(hcat(v))
#     Hv = I - 2*(vv * (vv'))
#     return Hv
# end

function house2(x::Vector{T}) where T<:Real
    m = length(x)
    v = [1 ; x[2:end]]
    v[1] = v[1] - norm(x)
    return return v./v[1]
    
end

function house(x::Vector{T}) where T<:Real
    m = length(x)
    σ = norm(x[2:end])^2
    v= [1 ; x[2:end]]
    if σ ≈ zero(σ) && x[1] ≥ 0
        β = 0
    elseif σ ≈ 0
        β = -2
    else 
        μ = sqrt(x[1]^2+σ)
        if x[1] ≤ 0
            v[1] = x[1] - μ
        else 
            v[1] = -σ/(x[1]+μ)
        end
        β = 2*v[1]^2 / (σ + v[1]^2)
        v=v./(v[1])
    end
    return β, v
end

function householder2(x::Vector{T}) where T<:Real
    m = length(x)
    v = [x[1] - norm(x); x[2:end]]
    vn = normalize(v)
    Hv = I - 2.0*v*v'
    return Hv*x
end

function householder(x::Vector{T}) where T<:Real
    β, v = house(x)
    return I - β*v*v'
end

function qrdecom(mat::Matrix{T}) where T<:Real
    
    m, n = size(mat)[1:2]
    @assert m == n
    
    if T<:Integer 
        A = Float64.(mat[:, :])
    else 
        A = mat[:, :]
    end

    elt = eltype(A)
    Q = Matrix{elt}(I, n, n)
    for i in 1:(n)
        Hm = householder(A[i:end, i])
        if i == 1 
            Q=Hm
            A = Q*A
        else
            X= [Matrix{elt}(I, i-1, i-1) zeros(elt, i-1, n-i+1); zeros(elt, i-1, n-i+1)'  Hm] 
            Q = X* Q
            A = X*A
        end
        
    end
    return Q', A
end

qrdecom (generic function with 1 method)

In [41]:
v = [1.0 2 1 1; 0 1 -1 5; 0 1 0 1; 1 0 1 -1]

4×4 Matrix{Float64}:
 1.0  2.0   1.0   1.0
 0.0  1.0  -1.0   5.0
 0.0  1.0   0.0   1.0
 1.0  0.0   1.0  -1.0

In [43]:
Q, R=qrdecom(v)


([0.7071067811865475 0.5000000000000001 0.2886751345948128 1.224744871391589; 0.0 0.5 -0.8660254037844384 1.0772401695030607e-16; 0.0 0.4999999999999999 0.288675134594813 -2.449489742783178; 0.7071067811865477 -0.5000000000000003 -0.28867513459481287 -1.2247448713915892], [1.4142135623730951 1.414213562373095 1.4142135623730951 -2.220446049250313e-16; -1.5700924586837752e-16 2.0 -0.5000000000000001 4.000000000000001; -9.064933036736784e-17 2.4467881385637166e-16 0.8660254037844383 -3.464101615137753; -3.8459253727671266e-16 -1.6714849754424513e-17 -4.4079092433785307e-16 -1.004422821527295e-15])

In [46]:
v .- Q*R

4×4 Matrix{Float64}:
  5.55112e-16  0.0           8.88178e-16  -2.22045e-16
  1.14005e-31  2.22045e-16  -5.55112e-16   1.77636e-15
 -8.37383e-16  1.11022e-16  -1.05074e-15  -2.66454e-15
 -6.66134e-16  4.9425e-16   -1.11022e-15   8.88178e-16

In [228]:
qq = qr(v)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 4×4 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
4×4 Matrix{Float64}:
 -1.41421  -1.41421  -1.41421    3.33067e-16
  0.0      -2.0       0.5       -4.0
  0.0       0.0      -0.866025   3.4641
  0.0       0.0       0.0       -4.44089e-16

In [230]:
Matrix(qq.Q)

4×4 Matrix{Float64}:
 -0.707107  -0.5  -0.288675  -0.408248
  0.0       -0.5   0.866025  -1.11022e-16
  0.0       -0.5  -0.288675   0.816497
 -0.707107   0.5   0.288675   0.408248

In [214]:
qq.Q*v

4-element Vector{Float64}:
 -2.613125929752753
 -0.0
 -0.0
  0.0

In [31]:
I[1:length(v), 1]

3-element Vector{Bool}:
 1
 0
 0

In [29]:
zero(1.0)

0.0

In [120]:
V = Matrix{Float64}(I, 1, 1)
X = [2.0 -1.0 2.0; -1.0 3.0 5.0; -1.0 -1.0 9.0]
h = [2.0 3.0 4.0]
[V h;h' X]

4×4 Matrix{Float64}:
 1.0   2.0   3.0  4.0
 2.0   2.0  -1.0  2.0
 3.0  -1.0   3.0  5.0
 4.0  -1.0  -1.0  9.0

In [118]:
h

1×1 Matrix{Float64}:
 1.0

In [163]:
v=[1,2,3]


3-element Vector{Int64}:
 1
 2
 3

In [16]:
x1 =[1.0, 0.0, 0.0, 0.0]
house(x1), house2(x1)

((0, [1.0, 0.0, 0.0, 0.0]), [0.0, 0.0, 0.0, 0.0])

In [17]:
householder(house2(x1))

4×4 Matrix{Float64}:
  1.0  -0.0  -0.0  -0.0
 -0.0   1.0  -0.0  -0.0
 -0.0  -0.0   1.0  -0.0
 -0.0  -0.0  -0.0   1.0

In [18]:
house([0,0,0.0])

(0, [1.0, 0.0, 0.0])

In [20]:
house2([-1,0,0.0])

3-element Vector{Float64}:
 0.0
 0.0
 0.0

In [21]:
house2([0,0,0.0])

3-element Vector{Float64}:
 1.0
 0.0
 0.0

In [25]:
householder(house2([-1,0,0.0]))

3×3 Matrix{Float64}:
  1.0  -0.0  -0.0
 -0.0   1.0  -0.0
 -0.0  -0.0   1.0

In [27]:
xx=[-1.0, 0, 0, 0]
house(xx), house2(xx)

((-2, [1.0, 0.0, 0.0, 0.0]), [0.0, 0.0, 0.0, 0.0])

In [29]:
householder(house(xx)[2])

4×4 Matrix{Float64}:
  1.0  -0.0  -0.0  -0.0
 -0.0   1.0  -0.0  -0.0
 -0.0  -0.0   1.0  -0.0
 -0.0  -0.0  -0.0   1.0

In [36]:
vv = [-1, 0, 0, 0]
house(vv)[2], house2(vv)

([1, 0, 0, 0], [NaN, NaN, NaN, NaN])

In [39]:
householder2(vv)

4-element Vector{Float64}:
 7.0
 0.0
 0.0
 0.0